## CNN-RNN文本分类

In [1]:
# coding=utf-8
import numpy as np
import itertools
import os

import tensorflow as tf

import time
import datetime

from tensorflow.contrib import learn
from tensorflow.contrib import rnn
import csv

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1.模型参数

In [2]:
# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("file_path", "thu_data_3class_3k", "Data source.")
tf.flags.DEFINE_integer("num_classes", 3, "number classes of datasets.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 200, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_integer("max_pool_size", 4, "max pool size")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")

tf.flags.DEFINE_integer("hidden_units", 200, "number of RNN hidden cell")
tf.flags.DEFINE_string("cell", 'lstm', "Which RNN cell will be used (dedault: lstm)")
tf.flags.DEFINE_float("num_rnn_layer", 1, "RNN layers (default: 1)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print('\nParameters')
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CELL=lstm
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=200
EVALUATE_EVERY=100
FILE_PATH=thu_data_3class_3k
FILTER_SIZES=3,4,5
HIDDEN_UNITS=200
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
MAX_POOL_SIZE=4
NUM_CHECKPOINTS=5
NUM_CLASSES=3
NUM_EPOCHS=100
NUM_FILTERS=200
NUM_RNN_LAYER=1



#### 2. 数据加载

In [3]:
import Ipynb_impoter
import data_helper

importing Jupyter notebook from data_helper.ipynb


In [4]:
input_x, input_y, words_index = data_helper.load_data(FLAGS.file_path, FLAGS.num_classes, 400)
x_train, x_dev, y_train, y_dev = train_test_split(input_x, input_y, train_size=0.9, random_state=123)
print("Vocabulary Size: {:d}".format(len(words_index)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 113453
Train/Dev split: 8100/900


#### 3.CNN-RNN模型

In [5]:
class TextCNNRNN(object):

    def __init__(self, embedding_mat,vocab_size, non_static, hidden_unit, sequence_length, max_pool_size,
                 num_classes, embedding_size, filter_sizes, num_filters, cell='lstm', num_layers=1, l2_reg_lambda=0.0):

        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        self.batch_size = tf.placeholder(tf.int32, [])
        self.pad = tf.placeholder(tf.float32, [None, 1, embedding_size, 1], name='pad')
        self.real_len = tf.placeholder(tf.int32, [None], name='real_len')
        self.cell = cell
        self.num_layers = num_layers

        l2_loss = tf.constant(0.0)

        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            if not non_static:
                W = tf.constant(embedding_mat, name='W')
            else:
                #W = tf.Variable(embedding_mat, name='W')
                W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1., 1.),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            emb = tf.expand_dims(self.embedded_chars, -1)

        pooled_concat = []
        reduced = np.int32(np.ceil((sequence_length) * 1.0 / max_pool_size))

        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope('conv-maxpool-%s' % filter_size):
                # Zero paddings so that the convolution output have dimension batch x sequence_length x emb_size x channel
                num_prio = (filter_size - 1) // 2
                num_post = (filter_size - 1) - num_prio
                pad_prio = tf.concat([self.pad] * num_prio, 1)
                pad_post = tf.concat([self.pad] * num_post, 1)
                emb_pad = tf.concat([pad_prio, emb, pad_post], 1)

                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name='b')
                conv = tf.nn.conv2d(emb_pad, W, strides=[1, 1, 1, 1], padding='VALID', name='conv')

                h = tf.nn.relu(tf.nn.bias_add(conv, b), name='relu')

                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, max_pool_size, 1, 1], strides=[1, max_pool_size, 1, 1], padding='SAME',
                                        name='pool')
                pooled = tf.reshape(pooled, [-1, reduced, num_filters])
                pooled_concat.append(pooled)

        pooled_concat = tf.concat(pooled_concat, 2)
        pooled_concat = tf.nn.dropout(pooled_concat, self.dropout_keep_prob)

        if self.cell == 'lstm':
            lstm_cell = rnn.BasicLSTMCell(num_units=hidden_unit)
        elif self.cell == 'gru':
            lstm_cell = rnn.GRUCell(num_units=hidden_unit)

        # add avg dropout at each layer
        if self.dropout_keep_prob is not None:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=self.dropout_keep_prob)

        self._initial_state = lstm_cell.zero_state(self.batch_size, tf.float32)
        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, reduced, pooled_concat)]
        inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(pooled_concat, num_or_size_splits=int(reduced), axis=1)]
        # outputs, state = tf.nn.rnn(lstm_cell, inputs, initial_state=self._initial_state, sequence_length=self.real_len)
        outputs, state = tf.contrib.rnn.static_rnn(lstm_cell, inputs, initial_state=self._initial_state,
                                                   sequence_length=self.real_len)

        # Collect the appropriate last words into variable output (dimension = batch x embedding_size)
        output = outputs[0]
        with tf.variable_scope('Output'):
            tf.get_variable_scope().reuse_variables()
            one = tf.ones([1, hidden_unit], tf.float32)
            for i in range(1, len(outputs)):
                ind = self.real_len < (i + 1)
                ind = tf.to_float(ind)
                ind = tf.expand_dims(ind, -1)
                mat = tf.matmul(ind, one)
                output = tf.add(tf.multiply(output, mat), tf.multiply(outputs[i], 1.0 - mat))

        with tf.name_scope('output'):
            self.W = tf.Variable(tf.truncated_normal([hidden_unit, num_classes], stddev=0.1), name='W')
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name='b')
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(output, self.W, b, name='scores')
            self.predictions = tf.argmax(self.scores, 1, name='predictions')

        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits(labels=self.input_y,
                                                             logits=self.scores)  # only named arguments accepted
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name='accuracy')

        with tf.name_scope('num_correct'):
            correct = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.num_correct = tf.reduce_sum(tf.cast(correct, 'float'))

#### 4. 模型训练

In [9]:
# Create a directory, everything related to the training will be saved in this directory
timestamp = str(int(time.time()))
trained_dir = './trained_results_' + timestamp + '/'
if os.path.exists(trained_dir):
    shutil.rmtree(trained_dir)
os.makedirs(trained_dir)

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn_rnn = TextCNNRNN(
            embedding_mat=None,
            vocab_size=len(words_index),
            sequence_length=x_train.shape[1],
            num_classes = FLAGS.num_classes,
            non_static=True,
            hidden_unit=FLAGS.hidden_units,
            max_pool_size=FLAGS.max_pool_size,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            embedding_size=FLAGS.embedding_dim,
            cell='lstm',
            num_layers=1,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        global_step = tf.Variable(0, name='global_step', trainable=False)
        optimizer = tf.train.RMSPropOptimizer(1e-3, decay=0.9)
        grads_and_vars = optimizer.compute_gradients(cnn_rnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Checkpoint files will be saved in this directory during training
        checkpoint_dir = './checkpoints_' + timestamp + '/'
        if os.path.exists(checkpoint_dir):
            shutil.rmtree(checkpoint_dir)
        os.makedirs(checkpoint_dir)
        checkpoint_prefix = os.path.join(checkpoint_dir, 'model')

        def real_len(batches):
            return [np.ceil(np.argmin(batch + [0]) * 1.0 / FLAGS.max_pool_size) for batch in batches]

        def train_step(x_batch, y_batch):
            feed_dict = {
                cnn_rnn.input_x: x_batch,
                cnn_rnn.input_y: y_batch,
                cnn_rnn.dropout_keep_prob: FLAGS.dropout_keep_prob,
                cnn_rnn.batch_size: len(x_batch),
                cnn_rnn.pad: np.zeros([len(x_batch), 1, FLAGS.embedding_dim, 1]),
                cnn_rnn.real_len: real_len(x_batch),
            }
            _, step, loss, accuracy = sess.run([train_op, global_step, cnn_rnn.loss, cnn_rnn.accuracy], feed_dict)

            time_str = datetime.datetime.now().isoformat()
            if step % 20 == 0:
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        def dev_step(x_batch, y_batch):
            feed_dict = {
                cnn_rnn.input_x: x_batch,
                cnn_rnn.input_y: y_batch,
                cnn_rnn.dropout_keep_prob: 1.0,
                cnn_rnn.batch_size: len(x_batch),
                cnn_rnn.pad: np.zeros([len(x_batch), 1, FLAGS.embedding_dim, 1]),
                cnn_rnn.real_len: real_len(x_batch),
            }
            step, loss, accuracy, num_correct, predictions = sess.run(
                [global_step, cnn_rnn.loss, cnn_rnn.accuracy, cnn_rnn.num_correct, cnn_rnn.predictions], feed_dict)
            
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            return accuracy, loss, num_correct, predictions

        saver = tf.train.Saver(tf.all_variables())
        sess.run(tf.initialize_all_variables())

        # Training starts here
        train_batches = data_helper.batch_iter(list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        best_accuracy, best_at_step = 0, 0

        # Train the model with x_train and y_train
        for train_batch in train_batches:
            x_train_batch, y_train_batch = zip(*train_batch)
            train_step(x_train_batch, y_train_batch)
            current_step = tf.train.global_step(sess, global_step)

            # Evaluate the model with x_dev and y_dev
            if current_step % FLAGS.checkpoint_every == 0:
                dev_batches = data_helper.batch_iter(list(zip(x_dev, y_dev)), FLAGS.batch_size, 1)

                total_dev_correct = 0
                for dev_batch in dev_batches:
                    x_dev_batch, y_dev_batch = zip(*dev_batch)
                    acc, loss, num_dev_correct, predictions = dev_step(x_dev_batch, y_dev_batch)
                　    total_dev_correct += num_dev_correct
                accuracy = float(total_dev_correct) / len(y_dev)
                print('Accuracy on dev set: {}'.format(accuracy))
                

                if accuracy >= best_accuracy:
                    best_accuracy, best_at_step = accuracy, current_step
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print('Saved model {} at step {}'.format(path, best_at_step))
                    print('Best accuracy {} at step {}'.format(best_accuracy, best_at_step))
        print('Training is complete, testing the best model on x_test and y_test')
        
        # Save the model files to trained_dir. predict.py needs trained model files.
        saver.save(sess, trained_dir + "best_model.ckpt")

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
2017-12-12T01:05:12.954478: step 20, loss 1.13049, acc 0.46875
2017-12-12T01:05:31.965224: step 40, loss 1.1605, acc 0.46875
2017-12-12T01:05:50.364293: step 60, loss 1.29808, acc 0.34375
2017-12-12T01:06:08.182127: step 80, loss 1.20596, acc 0.34375
2017-12-12T01:06:25.859321: step 100, loss 1.06597, acc 0.4375
2017-12-12T01:06:26.784871: step 100, loss 0.929247, acc 0.65625
2017-12-12T01:06:27.068394: step 100, loss 1.07424, acc 0.40625
2017-12-12T01:06:27.272912: step 100, loss 1.02029, acc 0.5
2017-12-12T01:06:27.493746: step 100, loss 0.855089, acc 0.6875
2017-12-12T01:06:27.961365: step 100, loss 0.971541, acc 0.5625
2017-12-12T01:06:28.170564: step 100, loss 0.967694, acc 0.46875
2017-12-12T01:06:28.385425: step 100, loss 0.937797, acc 0.625
2017-12-12T01:06:28.589758: step 100, loss 1.18774, acc 0.21875
2017-12-12T01:06:29.018850: step 100

2017-12-12T01:11:10.637937: step 400, loss 0.734091, acc 0.6875
2017-12-12T01:11:10.833031: step 400, loss 0.633975, acc 0.78125
2017-12-12T01:11:11.024665: step 400, loss 0.831819, acc 0.5625
2017-12-12T01:11:11.222987: step 400, loss 0.569288, acc 0.71875
2017-12-12T01:11:11.418465: step 400, loss 0.597919, acc 0.71875
2017-12-12T01:11:11.612399: step 400, loss 0.564087, acc 0.78125
2017-12-12T01:11:11.806271: step 400, loss 0.651682, acc 0.6875
2017-12-12T01:11:12.003517: step 400, loss 0.67126, acc 0.6875
2017-12-12T01:11:12.198466: step 400, loss 0.750123, acc 0.59375
2017-12-12T01:11:12.394379: step 400, loss 0.657871, acc 0.6875
2017-12-12T01:11:12.593087: step 400, loss 0.817541, acc 0.5
2017-12-12T01:11:12.784666: step 400, loss 0.855351, acc 0.5625
2017-12-12T01:11:12.992799: step 400, loss 0.71079, acc 0.6875
2017-12-12T01:11:13.193069: step 400, loss 0.654468, acc 0.78125
2017-12-12T01:11:13.391673: step 400, loss 0.724941, acc 0.59375
2017-12-12T01:11:13.590587: step 400, 

2017-12-12T01:14:56.266712: step 700, loss 0.458379, acc 0.75
Accuracy on dev set: 0.8822222222222222
Saved model ./checkpoints_1513011859/model-700 at step 700
Best accuracy 0.8822222222222222 at step 700
2017-12-12T01:15:14.015319: step 720, loss 0.126962, acc 0.96875
2017-12-12T01:15:27.100433: step 740, loss 0.371525, acc 0.8125
2017-12-12T01:15:40.019743: step 760, loss 0.253086, acc 0.90625
2017-12-12T01:15:52.472005: step 780, loss 0.295949, acc 0.84375
2017-12-12T01:16:05.667103: step 800, loss 0.524728, acc 0.84375
2017-12-12T01:16:05.870379: step 800, loss 0.20964, acc 0.90625
2017-12-12T01:16:06.067907: step 800, loss 0.320027, acc 0.875
2017-12-12T01:16:06.268065: step 800, loss 0.281073, acc 0.875
2017-12-12T01:16:06.464010: step 800, loss 0.183519, acc 0.96875
2017-12-12T01:16:06.658574: step 800, loss 0.282215, acc 0.875
2017-12-12T01:16:06.852969: step 800, loss 0.312925, acc 0.8125
2017-12-12T01:16:07.047650: step 800, loss 0.195293, acc 0.90625
2017-12-12T01:16:07.237

2017-12-12T01:19:45.863045: step 1100, loss 0.272695, acc 0.90625
2017-12-12T01:19:46.057577: step 1100, loss 0.0629394, acc 0.96875
2017-12-12T01:19:46.258455: step 1100, loss 0.0273331, acc 1
2017-12-12T01:19:46.463210: step 1100, loss 0.414758, acc 0.90625
2017-12-12T01:19:46.665084: step 1100, loss 0.111584, acc 0.9375
2017-12-12T01:19:46.860294: step 1100, loss 0.209926, acc 0.84375
2017-12-12T01:19:47.056386: step 1100, loss 0.502463, acc 0.84375
2017-12-12T01:19:47.257129: step 1100, loss 0.353493, acc 0.9375
2017-12-12T01:19:47.463953: step 1100, loss 0.411394, acc 0.90625
2017-12-12T01:19:47.664374: step 1100, loss 0.81779, acc 0.78125
2017-12-12T01:19:47.860158: step 1100, loss 0.282515, acc 0.90625
2017-12-12T01:19:48.053673: step 1100, loss 0.13736, acc 0.96875
2017-12-12T01:19:48.250758: step 1100, loss 0.574097, acc 0.875
2017-12-12T01:19:48.312915: step 1100, loss 0.00439191, acc 1
Accuracy on dev set: 0.9277777777777778
Saved model ./checkpoints_1513011859/model-1100 at

2017-12-12T01:24:37.911942: step 1500, loss 0.0290594, acc 1
2017-12-12T01:24:38.118656: step 1500, loss 0.108298, acc 0.96875
2017-12-12T01:24:38.320313: step 1500, loss 0.172681, acc 0.9375
2017-12-12T01:24:38.521763: step 1500, loss 0.171864, acc 0.9375
2017-12-12T01:24:38.718863: step 1500, loss 0.00240371, acc 1
2017-12-12T01:24:38.916480: step 1500, loss 0.0241119, acc 1
2017-12-12T01:24:39.116960: step 1500, loss 0.0239864, acc 1
2017-12-12T01:24:39.317552: step 1500, loss 0.0705754, acc 0.96875
2017-12-12T01:24:39.511866: step 1500, loss 0.00391401, acc 1
2017-12-12T01:24:39.711558: step 1500, loss 0.222158, acc 0.9375
2017-12-12T01:24:39.910304: step 1500, loss 0.231362, acc 0.9375
2017-12-12T01:24:40.103684: step 1500, loss 0.258568, acc 0.90625
2017-12-12T01:24:40.300017: step 1500, loss 0.0543399, acc 0.96875
2017-12-12T01:24:40.502963: step 1500, loss 0.0439002, acc 0.96875
2017-12-12T01:24:40.700661: step 1500, loss 0.127382, acc 0.96875
2017-12-12T01:24:40.897513: step 1

...

2017-12-12T05:40:25.486470: step 23800, loss 3.72529e-08, acc 1
2017-12-12T05:40:25.687675: step 23800, loss 4.84287e-08, acc 1
2017-12-12T05:40:25.866368: step 23800, loss 0.665293, acc 0.96875
2017-12-12T05:40:26.058667: step 23800, loss 0.00428112, acc 1
2017-12-12T05:40:26.256500: step 23800, loss 8.6755e-06, acc 1
2017-12-12T05:40:26.454945: step 23800, loss 0.00323924, acc 1
2017-12-12T05:40:26.656937: step 23800, loss 0, acc 1
2017-12-12T05:40:26.854116: step 23800, loss 0, acc 1
2017-12-12T05:40:27.055841: step 23800, loss 0.668879, acc 0.90625
2017-12-12T05:40:27.248888: step 23800, loss 1.59168, acc 0.90625
2017-12-12T05:40:27.441814: step 23800, loss 7.45058e-09, acc 1
2017-12-12T05:40:27.636490: step 23800, loss 1.1311, acc 0.9375
2017-12-12T05:40:27.830859: step 23800, loss 0.104239, acc 0.96875
2017-12-12T05:40:28.024464: step 23800, loss 2.36714, acc 0.875
2017-12-12T05:40:28.220772: step 23800, loss 0.799891, acc 0.9375
2017-12-12T05:40:28.410044: step 23800, loss 0.566

2017-12-12T05:43:53.352460: step 24100, loss 0.741641, acc 0.96875
2017-12-12T05:43:53.542872: step 24100, loss 1.09923, acc 0.9375
2017-12-12T05:43:53.734887: step 24100, loss 7.45058e-09, acc 1
2017-12-12T05:43:53.769698: step 24100, loss 0.000284301, acc 1
Accuracy on dev set: 0.9655555555555555
2017-12-12T05:44:06.142238: step 24120, loss 0, acc 1
2017-12-12T05:44:18.088173: step 24140, loss 0, acc 1
2017-12-12T05:44:30.592790: step 24160, loss 1.71363e-07, acc 1
2017-12-12T05:44:42.966953: step 24180, loss 0, acc 1
2017-12-12T05:44:55.370517: step 24200, loss 1.11759e-08, acc 1
2017-12-12T05:44:55.569665: step 24200, loss 0.147837, acc 0.96875
2017-12-12T05:44:55.764350: step 24200, loss 0, acc 1
2017-12-12T05:44:55.966787: step 24200, loss 0, acc 1
2017-12-12T05:44:56.165154: step 24200, loss 0.86734, acc 0.96875
2017-12-12T05:44:56.362056: step 24200, loss 0.412328, acc 0.96875
2017-12-12T05:44:56.562575: step 24200, loss 0.00790973, acc 1
2017-12-12T05:44:56.757052: step 24200,

2017-12-12T05:48:21.266620: step 24500, loss 0.612738, acc 0.96875
2017-12-12T05:48:21.458877: step 24500, loss 0.348253, acc 0.9375
2017-12-12T05:48:21.653928: step 24500, loss 0.775817, acc 0.96875
2017-12-12T05:48:21.848830: step 24500, loss 0.370836, acc 0.96875
2017-12-12T05:48:22.044023: step 24500, loss 0.452348, acc 0.96875
2017-12-12T05:48:22.242657: step 24500, loss 0.344966, acc 0.9375
2017-12-12T05:48:22.437621: step 24500, loss 0.417478, acc 0.96875
2017-12-12T05:48:22.635014: step 24500, loss 3.05023e-05, acc 1
2017-12-12T05:48:22.824585: step 24500, loss 0.0510577, acc 0.96875
2017-12-12T05:48:23.020840: step 24500, loss 0.545082, acc 0.96875
2017-12-12T05:48:23.216082: step 24500, loss 0.346712, acc 0.9375
2017-12-12T05:48:23.256970: step 24500, loss 0, acc 1
Accuracy on dev set: 0.9655555555555555
2017-12-12T05:48:35.697107: step 24520, loss 0.0541254, acc 0.96875
2017-12-12T05:48:48.133098: step 24540, loss 0, acc 1
2017-12-12T05:49:00.686404: step 24560, loss 0, acc 

2017-12-12T05:52:49.805717: step 24900, loss 0, acc 1
2017-12-12T05:52:49.993127: step 24900, loss 0.857814, acc 0.9375
2017-12-12T05:52:50.189390: step 24900, loss 4.84287e-08, acc 1
2017-12-12T05:52:50.390639: step 24900, loss 0.00095359, acc 1
2017-12-12T05:52:50.585465: step 24900, loss 0, acc 1
2017-12-12T05:52:50.780379: step 24900, loss 1.06196e-05, acc 1
2017-12-12T05:52:50.976015: step 24900, loss 2.41389e-06, acc 1
2017-12-12T05:52:51.171392: step 24900, loss 0.865931, acc 0.9375
2017-12-12T05:52:51.364693: step 24900, loss 0.878283, acc 0.90625
2017-12-12T05:52:51.561425: step 24900, loss 0.141444, acc 0.96875
2017-12-12T05:52:51.758508: step 24900, loss 0, acc 1
2017-12-12T05:52:51.954207: step 24900, loss 0.989525, acc 0.84375
2017-12-12T05:52:52.153327: step 24900, loss 3.4645e-07, acc 1
2017-12-12T05:52:52.346664: step 24900, loss 0.282762, acc 0.9375
2017-12-12T05:52:52.541046: step 24900, loss 2.92795e-06, acc 1
2017-12-12T05:52:52.730394: step 24900, loss 0.020435, ac

2017-12-12T05:57:18.082657: step 25300, loss 0.864463, acc 0.9375
2017-12-12T05:57:18.278142: step 25300, loss 0, acc 1
2017-12-12T05:57:18.472171: step 25300, loss 7.45058e-09, acc 1
2017-12-12T05:57:18.663868: step 25300, loss 0.569216, acc 0.9375
2017-12-12T05:57:18.855945: step 25300, loss 0.847354, acc 0.9375
2017-12-12T05:57:19.047126: step 25300, loss 2.08286e-05, acc 1
2017-12-12T05:57:19.243350: step 25300, loss 0.513723, acc 0.96875
2017-12-12T05:57:19.438320: step 25300, loss 0.00689045, acc 1
2017-12-12T05:57:19.632967: step 25300, loss 1.4205, acc 0.9375
2017-12-12T05:57:19.832963: step 25300, loss 0.676764, acc 0.96875
2017-12-12T05:57:20.016770: step 25300, loss 0.351332, acc 0.9375
2017-12-12T05:57:20.212386: step 25300, loss 1.12063, acc 0.90625
2017-12-12T05:57:20.403077: step 25300, loss 0.00661484, acc 1
2017-12-12T05:57:20.607109: step 25300, loss 0.110641, acc 0.9375
2017-12-12T05:57:20.802489: step 25300, loss 0.0140085, acc 1
2017-12-12T05:57:20.996591: step 253